<h2>IMDB dateset</h2>

IMDB는 (영화 리뷰, 리뷰의 긍정부정 값)을 한 쌍의 데이터 값으로, 50,000개의 샘플을 모아 놓은 데이터 셋이다. 이 중 25,000개의 댓글 셈플은 트레이닝 샘플이며, 나머지 25,000개의 댓글 샘플은 테스팅 샘플이다.<br>
다시말해, 해당 데이터 셋에는 50000개의 영화 리뷰를 가지고 있으며, 모든 리뷰들은 결과 값(부정 혹은 긍적) 값을 결과 값으로 가지고 있다. 
해당 데이터 셋의 결과 값이 이진 데이터라는 특성 때문에 해당 데이터 셋을 사용하는 신경망 모델의 목적은 주로 binary classification을 목적으로 한다.

해당 파일에서는 편의를 위해 데이터를 로드할 때, 가장 빈번하게 사용되는 20,000개의 단어들만 포함하여 로드하였다. 

In [1]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=20000)

Using TensorFlow backend.


모델 내부에서 training을 위한 데이터와 이를 각 epoch에서 검증하기 위한 데이터를 분류
하기 위해 25,000개의 데이터를 분할하였다.<br>
25,000개이 샘플 중 20,000개는 트레이닝을 위한 샘플, 나머지 5,000개는 트레이닝 후 정확도 평가를 위한 validation 샘플로 두었다.

In [2]:
x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]

<h4>pad_sequence() function</h4>
pad_sequence()라는 함수는 케라스에 내장되어 있는 함수로, 데이터 전처리를 하는 것이 그 용도이다. </br>
해당 함수를 사용하면, 샘플 간의 길이를 균등하게 맞출 수 있다.

In [3]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=200)
x_val = sequence.pad_sequences(x_val, maxlen=200)
x_test = sequence.pad_sequences(x_test, maxlen=200)

<h2>모델 구성</h2>
Sequential() 함수로 모델을 초기화 한다.

<h4>Embedding()층을 추가한다</h4>
Embedding()은 one-hot-encoding보다 훨씬 적은 공간을 사용한다. 또한 해당 층을 추가하면 단어 간의 연관성을 고려한 벡터가 만들어지기 때문에 훨씬 유의미한 훈련이 가능하다.

<h4>LSTM</h4>
LSTM은 Long Short-term Memory의 약자이다. 이는 기본적인 RNN 신경망 구조에서 변형된 형태이다. 기존의 RNN은 입력값의 시퀀스 길이가 길어질 때, 훈련이 반영이 잘 되지 않는 "Vanishing gradient"문제 떄문에 해당 모델이 구상되었다.<br>
해당 모델은 훈련이 거듭될 때마다 결과 값을 기억하고 있는다. 다만, 새로운 입력값과 기억하고 있는 데이터의 연관 정도를 고려하여 기억 데이터의 일부분을 삭제하고 새로운 입력 값을 기억한다는 적이 해당 기법의 특이한 점이다. 

<h4>출력층: Sigmoid 함수</h4>
해당 모델의 목적은 입력되는 리뷰의 긍정 부정을 판별하는 것이다. Binary classification에 적합한 Sigmoid 함수를 사용하였다.


<h4>Adam optimizer</h4>
<p>
    ADAM stands for Adaptive Moment Estimation
ADAM is gradient descent using both Momentum gradient descent and RMprop gradient descent.<br></p>
    <h4>Argument</h4>
<p>
lr: float >= 0. Learning rate.<br>
beta_1: float, 0 < beta < 1. Generally close to 1.<br>
beta_2: float, 0 < beta < 1. Generally close to 1.<br>
epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().<br>
decay: float >= 0. Learning rate decay over each update.<br>
amsgrad: boolean. Whether to apply the AMSGrad variant of this algorithm from the paper "On the Convergence of Adam and Beyond".<br>
</p>

In [4]:
from keras import models 
from keras.layers import Embedding, Dense, LSTM

model = models.Sequential()
model.add(Embedding(20000, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

hist = model.fit(x_train, y_train, epochs=25, batch_size=64, validation_data=(x_val, y_val))

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/25
20000/20000 [==============================] - 109s 5ms/step - loss: 0.4625 - acc: 0.7827 - val_loss: 0.4485 - val_acc: 0.7982
Epoch 2/25
20000/20000 [==============================] - 109s 5ms/step - loss: 0.2447 - 

In [15]:
result=model.evaluate(x_test, y_test)

25000/25000 [==============================] - 87s 3ms/step


In [19]:
print("====Test Score(loss)====")
print(result[0])
print("====Test Accuracy====")
print(result[1])

====Test Score(loss)====
1.445977435076018
====Test Accuracy====
0.8438
